In [1]:
import pandas as pd

In [4]:
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

In [5]:
#claim files for merging
claims1 = pd.read_csv('../../DE1_0_2008_to_2010_Carrier_Claims_Sample_2A.csv')

/Users/ksenia/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,10,11,46,47,48,49,50,136,137,138,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


KeyboardInterrupt: 

In [ ]:
claims2 = pd.read_csv('../../DE1_0_2008_to_2010_Carrier_Claims_Sample_2B.csv')

In [ ]:
#Merging two claims files
claims = pd.concat([claims1, claims2])

In [6]:
#Obtain inpatient claims
inpatient = pd.read_csv('../../DE1_0_2008_to_2010_Inpatient_Claims_Sample_2.csv')

In [ ]:
#Obtain outpatient claims data
outpatient = pd.read_csv('../../DE1_0_2008_to_2010_Outpatient_Claims_Sample_2.csv')

In [ ]:
#Obtain prescription drug data
pde = pd.read_csv('../../DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_2.csv')

In [7]:
#Beneficiary data files for merging
ben8 = pd.read_csv('../../DE1_0_2008_Beneficiary_Summary_File_Sample_2.csv')
ben9 = pd.read_csv('../../DE1_0_2009_Beneficiary_Summary_File_Sample_2.csv')
ben10 = pd.read_csv('../../DE1_0_2010_Beneficiary_Summary_File_Sample_2.csv')

In [8]:
ben_all = pd.concat([ben8, ben9, ben10])

In [9]:
#Select a random sample of patients for further analysis if sample=True

sample = False

if sample is True:
    sample_patient_ids = ben10.DESYNPUF_ID.sample(frac = 0.5)
else:
    sample_patient_ids = ben10.DESYNPUF_ID

In [10]:
ben10.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,00000B48BCF4AD29,19230901,NaN,2,5,Y,10,260,12,12,...,2,0.0,0.0,0.0,600.0,30.0,0.0,3800.0,1460.0,0.0
1,0000525AB30E4DEF,19201001,NaN,2,1,0,31,300,12,12,...,2,0.0,0.0,0.0,240.0,190.0,0.0,440.0,60.0,0.0
2,00009C897C3D8372,19320101,NaN,1,1,0,7,70,12,12,...,2,0.0,0.0,0.0,210.0,160.0,0.0,580.0,250.0,0.0
3,0001168CE43BE51B,19340901,NaN,2,1,0,6,200,12,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0002E494BC87CE10,19140701,NaN,1,2,0,5,200,12,12,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Write beneficiary features of interest for corresponding patients into a single dataframe for preprocessing

working_df = pd.DataFrame(index = ben10.index)

beneficiary_features = ['DESYNPUF_ID','BENE_SEX_IDENT_CD', 'BENE_RACE_CD', 'BENE_BIRTH_DT', 'SP_STATE_CODE', 
                        'BENE_COUNTY_CD','SP_ALZHDMTA', 'SP_CHF', 'SP_CHRNKIDN', 'SP_CNCR', 'SP_COPD', 'SP_DIABETES', 'SP_DEPRESSN', 
                        'SP_ISCHMCHT', 'SP_OSTEOPRS', 'SP_RA_OA', 'SP_STRKETIA', 'MEDREIMB_OP', 'BENRES_OP', 
                        'PPPYMT_OP', 'BENE_DEATH_DT']    

In [12]:
#Select beneficiary data corresponding to the sample size
working_df = ben10[ben10.DESYNPUF_ID.isin(sample_patient_ids)]


#Drop features that are not of interest
for col in working_df:
    print(col)
    if col not in beneficiary_features:
        working_df.drop(col, axis = 1, inplace = True)
        

DESYNPUF_ID
BENE_BIRTH_DT
BENE_DEATH_DT
BENE_SEX_IDENT_CD
BENE_RACE_CD
BENE_ESRD_IND
SP_STATE_CODE
BENE_COUNTY_CD
BENE_HI_CVRAGE_TOT_MONS
BENE_SMI_CVRAGE_TOT_MONS
BENE_HMO_CVRAGE_TOT_MONS
PLAN_CVRG_MOS_NUM
SP_ALZHDMTA
SP_CHF
SP_CHRNKIDN
SP_CNCR
SP_COPD
SP_DEPRESSN
SP_DIABETES
SP_ISCHMCHT
SP_OSTEOPRS
SP_RA_OA
SP_STRKETIA
MEDREIMB_IP
BENRES_IP
PPPYMT_IP
MEDREIMB_OP
BENRES_OP
PPPYMT_OP
MEDREIMB_CAR
BENRES_CAR
PPPYMT_CAR


In [13]:
#Drop rows and columns where all elements are NULL
working_df = working_df.dropna(axis=1, how='all')
working_df = working_df.dropna(axis=0, how='all')

In [14]:
working_df.describe()

,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,MEDREIMB_OP,BENRES_OP,PPPYMT_OP
count,1.128450e+05,1.818000e+03,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000,112845.000000
mean,1.936439e+07,2.010064e+07,1.557251,1.283708,25.725048,366.079268,1.831929,1.742124,1.861403,1.948212,1.910904,1.821188,1.708760,1.623918,1.868900,1.902849,1.973530,435.842882,131.627985,14.953875
std,1.258994e+05,3.460075e+02,0.496714,0.753121,15.519775,266.004435,0.373931,0.437467,0.345527,0.221599,0.284883,0.383196,0.454336,0.484403,0.337511,0.296165,0.160529,1280.079036,367.158699,274.898947
min,1.909010e+07,2.010010e+07,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-80.000000,0.000000,0.000000
25%,1.928110e+07,2.010030e+07,1.000000,1.000000,11.000000,141.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
50%,1.936060e+07,2.010060e+07,2.000000,1.000000,25.000000,330.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000
75%,1.942030e+07,2.010090e+07,2.000000,1.000000,39.000000,551.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,320.000000,100.000000,0.000000
max,1.983120e+07,2.010120e+07,2.000000,5.000000,54.000000,999.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,34440.000000,9670.000000,14000.000000


In [15]:
#Create a variable indicating if a patient has passed away
working_df['DEATH'] = working_df.BENE_DEATH_DT.apply(lambda x: 1 if pd.notnull(x) else 0)

In [16]:
#Add hospitalization date to the working dataframe
working_df = pd.merge(working_df, inpatient[['DESYNPUF_ID','CLM_ADMSN_DT']], how='left', on='DESYNPUF_ID')

In [17]:
#target_date - start date of forecasting patients hospitalization 
target_date = 20100101

In [18]:
#Create target variable Y showing 1 if patient has been hospitalized in the target dataframe
max_ip_date = working_df.CLM_ADMSN_DT.groupby(working_df.DESYNPUF_ID).max()
inp_dates = pd.DataFrame(list(max_ip_date.items()), columns = ['DESYNPUF_ID', 'CLM_ADMSN_DT'])
inp_dates['Y'] = inp_dates.CLM_ADMSN_DT.apply(lambda x: 1 if x >= target_date else 0)
working_df = pd.merge(working_df, inp_dates[['DESYNPUF_ID','Y']], how='left', on='DESYNPUF_ID')

In [19]:
#If hospitalized, first admission date in the target dataframe
dates = {}
sub_df = working_df[working_df.Y==1]

for patient in sub_df.DESYNPUF_ID.unique():
    hosp_dates = working_df[working_df.DESYNPUF_ID==patient].CLM_ADMSN_DT
    min_date_2010 = min(hosp_dates[hosp_dates>=20100101])
    dates[patient] = min_date_2010

In [20]:
#Adding first admission date to the dataframe
df_dates = pd.DataFrame(list(dates.items()), columns = ['DESYNPUF_ID', 'fst_admsn_dt'])
working_df = pd.merge(working_df, df_dates[['DESYNPUF_ID','fst_admsn_dt']], how='left', on='DESYNPUF_ID')

In [21]:
import datetime
from datetime import date
import dateutil
from datetime import datetime
from dateutil import parser
import numpy as np

def clean_df(df):
    
    '''Takes as input a DataFrame, creates a dummy variable for the death date (0 if still alive, 1 if not).
    Calculates the column TRUE_AGE, according to the death date if the dummy variable indicating death is 1, 
    or according to today's date if the dummy variable is 0. Recodes the column BENE_SEX_IDENT_CD (Sex) as 
    1: Female, 0: Male. '''
    
    days_a_year = 365.24
    
    ## Filling the missing values with 0, as explained above
    df['BENE_DEATH_DT'] = df['BENE_DEATH_DT'].fillna(0)
    
    ## Creating the column DEATH_DUMMY, 1 if the patient is dead, 0 otherwise
    df['DEATH_DUMMY'] = [int(int(df['BENE_DEATH_DT'][k])>0) for k in range(len(df['BENE_DEATH_DT']))]
    df['BENE_BIRTH_DT'] = [str(j) for j in df['BENE_BIRTH_DT']]
    
    df['BENE_DEATH_DT'] = [str(int("%2.f" % float(j))) for j in df['BENE_DEATH_DT']]
    
    bday_date = [dateutil.parser.parse(date) for date in df['BENE_BIRTH_DT']]
    
    death_date = []
    for d in range(len(df)):
        if df['DEATH_DUMMY'][d] == 0:
            death_date.append(dateutil.parser.parse(str(20100101)))
        elif df['DEATH_DUMMY'][d] == 1:
            death_date.append(dateutil.parser.parse(df['BENE_DEATH_DT'][d]))
    
    df['TRUE_AGE'] = [((death_date[j] - bday_date[j])/days_a_year).days for j in range(len(df['BENE_BIRTH_DT']))]
    
        ## Sex recoded to (0:Male, 1:Female) 
    df['BENE_SEX_IDENT_CD'] = [int(df['BENE_SEX_IDENT_CD'][i] == 2) for i in range(len(df['BENE_SEX_IDENT_CD']))]
    
    return df

In [22]:
clean_df = clean_df(working_df)

In [24]:
clean_df['first_ad_date']=clean_df.fst_admsn_dt.astype(str).apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))

In [25]:
import datetime
import datetime
import calendar

def add_months(sourcedate, months):
    mmonth = sourcedate.month - 1 + months
    yyear = int(sourcedate.year + mmonth / 12)
    mmonth = mmonth % 12 + 1
    dday = min(sourcedate.day,calendar.monthrange(yyear,mmonth)[1])
    return datetime.date(yyear,mmonth,dday)


clean_df.first_ad_date.fillna(datetime.date(3000,1,1), inplace = True)

In [26]:
clean_df[clean_df.Y==1].head(20)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,...,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,DEATH,CLM_ADMSN_DT,Y,fst_admsn_dt,DEATH_DUMMY,TRUE_AGE,first_ad_date
21,0008148FCB4CB644,19420601,0,0,3,14,141,2,1,1,...,17990.0,6770.0,0.0,0,20091211.0,1,20100113.0,0,67,2010-01-13 00:00:00
22,0008148FCB4CB644,19420601,0,0,3,14,141,2,1,1,...,17990.0,6770.0,0.0,0,20100113.0,1,20100113.0,0,67,2010-01-13 00:00:00
27,000B6900337D45AF,19490601,0,1,1,34,500,2,2,2,...,0.0,0.0,0.0,0,20100831.0,1,20100831.0,0,60,2010-08-31 00:00:00
30,000C5BACF2F88B1E,19420201,0,1,1,42,270,1,2,2,...,290.0,220.0,0.0,0,20090218.0,1,20100927.0,0,67,2010-09-27 00:00:00
31,000C5BACF2F88B1E,19420201,0,1,1,42,270,1,2,2,...,290.0,220.0,0.0,0,20091212.0,1,20100927.0,0,67,2010-09-27 00:00:00
32,000C5BACF2F88B1E,19420201,0,1,1,42,270,1,2,2,...,290.0,220.0,0.0,0,20100927.0,1,20100927.0,0,67,2010-09-27 00:00:00
44,001539F8694D461D,19390201,0,1,1,33,590,1,1,1,...,880.0,0.0,0.0,0,20100730.0,1,20100730.0,0,70,2010-07-30 00:00:00
46,001634033AEA61D0,19380901,0,0,1,10,150,1,2,2,...,0.0,0.0,0.0,0,20100126.0,1,20100126.0,0,71,2010-01-26 00:00:00
60,001D171C944A6B21,19310701,0,1,1,33,740,1,2,2,...,460.0,130.0,0.0,0,20080204.0,1,20100405.0,0,78,2010-04-05 00:00:00
61,001D171C944A6B21,19310701,0,1,1,33,740,1,2,2,...,460.0,130.0,0.0,0,20100405.0,1,20100405.0,0,78,2010-04-05 00:00:00


In [27]:
import datetime
clean_df.first_ad_date.fillna(datetime.date(3000,1,1), inplace = True)

In [28]:
clean_df['end_train_dt']=clean_df.first_ad_date.apply(lambda x: add_months(x,-6))
clean_df.end_train_dt.replace(datetime.date(2999,7,1),datetime.date(2010,6,30),inplace=True)

In [29]:
clean_df.CLM_ADMSN_DT.fillna(datetime.date(3000,1,1), inplace = True)

In [30]:
clean_df['start_train_dt']=datetime.date(2008,1,1)

In [31]:
def diff_month(d1, d2):
    return(d1.year - d2.year) * 12 + d1.month - d2.month

import math

date1 = datetime.date(2009, 7, 13)
date2 = datetime.date(2008, 1, 1)
date2

def month_difference(date1, date2):
    
    '''Takes as input two dates, takes their timedelta, if it is exactly x.5 months returns x.5 months,
    if it is between x and x.5 months returns x months, and if it is between x.5 months and ceiling(x),
    returns ceiling(x)'''
    
    date_diff_days = (date1 - date2).days ## timedelta.days, it doesn't have months or years methods unfortunately
    date_diff_months = (date_diff_days/30.4)
    
    if math.ceil(date_diff_months) - date_diff_months > 0.5:
        return math.trunc(date_diff_months)
    elif math.ceil(date_diff_months) - date_diff_months <= 0.5:
        return math.ceil(date_diff_months)
    else:
        return (math.trunc(date_diff_months) + 0.5) 
        

month_difference(date1, date2)

clean_df['member_months'] = [month_difference(clean_df['end_train_dt'][i], clean_df['start_train_dt'][i]) for i in range(len(clean_df))]

clean_df[clean_df.Y==1].head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,...,DEATH,CLM_ADMSN_DT,Y,fst_admsn_dt,DEATH_DUMMY,TRUE_AGE,first_ad_date,end_train_dt,start_train_dt,member_months
21,0008148FCB4CB644,19420601,0,0,3,14,141,2,1,1,...,0,2.00912e+07,1,20100113.0,0,67,2010-01-13 00:00:00,2009-07-13,2008-01-01,18
22,0008148FCB4CB644,19420601,0,0,3,14,141,2,1,1,...,0,2.01001e+07,1,20100113.0,0,67,2010-01-13 00:00:00,2009-07-13,2008-01-01,18
27,000B6900337D45AF,19490601,0,1,1,34,500,2,2,2,...,0,2.01008e+07,1,20100831.0,0,60,2010-08-31 00:00:00,2010-02-28,2008-01-01,26
30,000C5BACF2F88B1E,19420201,0,1,1,42,270,1,2,2,...,0,2.00902e+07,1,20100927.0,0,67,2010-09-27 00:00:00,2010-03-27,2008-01-01,27
31,000C5BACF2F88B1E,19420201,0,1,1,42,270,1,2,2,...,0,2.00912e+07,1,20100927.0,0,67,2010-09-27 00:00:00,2010-03-27,2008-01-01,27


In [32]:
#Previous inpatient hospitalization date (last date in the training period)

print(clean_df.shape)
print(inpatient.shape)

N = 100000000
clean_sub = clean_df.iloc[0:min(clean_df.shape[0],N),:]
inpat_sub = inpatient.iloc[0:min(inpatient.shape[0],N),:]

prev_hosp_dt = {}

print('figuring end dates')
dates = clean_sub.end_train_dt.groupby(clean_df.DESYNPUF_ID).max()
print('DONE figuring end dates')

def process_patient_rows(rows):
    global dates, prev_hosp_dt
    if len(prev_hosp_dt) % 1000 == 0:
        print(str(len(prev_hosp_dt)))
    pat_id = rows.DESYNPUF_ID.iloc[0]
    if pat_id not in dates:
        return
    last_training_date = dates[pat_id]

    training_selection = rows.CLM_ADMSN_DT.apply(
        lambda x: (dateutil.parser.parse(str(x))).date() <= last_training_date)
    last_hosp_date = rows.CLM_ADMSN_DT[training_selection].max()
    if not np.isnan(last_hosp_date):
        prev_hosp_dt[pat_id] = last_hosp_date
    
inpat_sub.groupby(inpat_sub.DESYNPUF_ID).apply(lambda rows: process_patient_rows(rows))
"""
for patient in inpat_sub.DESYNPUF_ID.unique():
    
    patient_rows = inpat_sub[inpat_sub.DESYNPUF_ID == patient]
"""
print(len(prev_hosp_dt))

(141363, 31)
(66494, 81)
figuring end dates
DONE figuring end dates
0
1000
1000
1000
1000
2000
2000
3000
4000
5000
6000
7000
8000
9000
9000
10000
10000
11000
11000
12000
12000
13000
14000
15000
16000
17000
18000
19000
20000
20000
21000
21000
22000
23000
24000
24000
24000
24000
24000
25000
25000
26000
27000
28000
28651


In [33]:
#Convert to date type and add to the data frame
prev_hosp = pd.DataFrame(list(prev_hosp_dt.items()), columns = ['DESYNPUF_ID', 'prev_hosp'])
prev_hosp['prev_hosp_dt']=prev_hosp.prev_hosp.astype(str).apply(lambda x: pd.to_datetime(x, format='%Y%m%d'))


In [34]:
#Add hospitalization date to the working dataframe
clean_df = pd.merge(clean_df, prev_hosp[['DESYNPUF_ID','prev_hosp_dt']], how='left', on='DESYNPUF_ID')

In [35]:
#Number of unique IP dates in training period
unique_ip = inpatient.CLM_ADMSN_DT.groupby(inpatient.DESYNPUF_ID).nunique()
unique_ip_dt = pd.DataFrame({'DESYNPUF_ID':unique_ip.index, 'unique_ip_dt':unique_ip.values})
clean_df = pd.merge(clean_df, unique_ip_dt[['DESYNPUF_ID','unique_ip_dt']], how='left', on='DESYNPUF_ID')

In [36]:
#Dollar spend on inpatient claims in training period
inpatient.head()

inpatient['CLM_THRU_DT'].fillna(30000101.0, inplace=True)

In [37]:
inpatient['CLM_THRU_DT'] = [str(int(x)) for x in inpatient['CLM_THRU_DT']]

In [38]:
inpatient['CLM_THRU_DT'] = [dateutil.parser.parse(str(date)).date() for date in inpatient['CLM_THRU_DT']]

In [39]:
inpatient.head()

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,00000B48BCF4AD29,45461150060934,1,20080707.0,2008-07-18,1000GD,57000.0,0.0,1.219278e+09,7.451661e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00000B48BCF4AD29,45541150055035,1,20080724.0,2008-07-26,1030RD,0.0,0.0,9.812675e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00000B48BCF4AD29,45731150068478,1,20080805.0,2008-08-20,1001NA,24000.0,0.0,6.035410e+09,9.404475e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000525AB30E4DEF,45811150057351,1,20080525.0,2008-06-05,3900MB,4000.0,0.0,1.511429e+09,7.340276e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000525AB30E4DEF,45391150058019,1,20080918.0,2008-09-24,3100JN,8000.0,0.0,3.198318e+09,9.156656e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
import dateutil

def calc_ip_amt(end_train_dt, patient_id):
    #end_train_dt = dateutil.parser.parse(str(end_train_dt)).date()
    #pd.to_datetime(x, format='%Y%m%d')
    id_mask = (inpatient.DESYNPUF_ID == patient_id)
    return inpatient[inpatient.CLM_THRU_DT<=end_train_dt][id_mask].CLM_PMT_AMT.sum()

pmts = {}

train_dates = clean_df.end_train_dt.groupby(clean_df.DESYNPUF_ID).min()

In [54]:
inpatient = pd.merge(inpatient, clean_df[['DESYNPUF_ID','end_train_dt']], how='left', on='DESYNPUF_ID')

In [56]:
inpatient['count_clm'] = inpatient.CLM_THRU_DT<inpatient.end_train_dt

In [70]:
agg = pd.pivot_table(inpatient,values 
               =['CLM_PMT_AMT'],index=['count_clm'],columns=['DESYNPUF_ID'],aggfunc=np.sum).transpose()

In [107]:
comp = pd.DataFrame(agg.to_records())

In [109]:
clean_df = pd.merge(clean_df, comp[['DESYNPUF_ID','True']],how='left',on='DESYNPUF_ID')

In [112]:
clean_df.rename(index=str, columns={"True": "IP_PMT"})

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,...,fst_admsn_dt,DEATH_DUMMY,TRUE_AGE,first_ad_date,end_train_dt,start_train_dt,member_months,prev_hosp_dt,unique_ip_dt,IP_PMT
0,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
1,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
2,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
3,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,NaN,0,89,3000-01-01,2010-06-30,2008-01-01,30,2008-09-18,2.0,24000.0
4,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,NaN,0,89,3000-01-01,2010-06-30,2008-01-01,30,2008-09-18,2.0,24000.0
5,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,NaN,0,78,3000-01-01,2010-06-30,2008-01-01,30,2008-09-21,5.0,180500.0
6,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,NaN,0,78,3000-01-01,2010-06-30,2008-01-01,30,2008-09-21,5.0,180500.0
7,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,NaN,0,78,3000-01-01,2010-06-30,2008-01-01,30,2008-09-21,5.0,180500.0
8,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,NaN,0,78,3000-01-01,2010-06-30,2008-01-01,30,2008-09-21,5.0,180500.0
9,00009C897C3D8372,19320101,0,0,1,7,70,2,1,2,...,NaN,0,78,3000-01-01,2010-06-30,2008-01-01,30,2008-09-21,5.0,180500.0


In [110]:
clean_df.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,SP_STATE_CODE,BENE_COUNTY_CD,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,...,fst_admsn_dt,DEATH_DUMMY,TRUE_AGE,first_ad_date,end_train_dt,start_train_dt,member_months,prev_hosp_dt,unique_ip_dt,True
0,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
1,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
2,00000B48BCF4AD29,19230901,0,1,5,10,260,2,1,2,...,NaN,0,86,3000-01-01,2010-06-30,2008-01-01,30,2008-08-05,3.0,243000.0
3,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,NaN,0,89,3000-01-01,2010-06-30,2008-01-01,30,2008-09-18,2.0,24000.0
4,0000525AB30E4DEF,19201001,0,1,1,31,300,2,2,2,...,NaN,0,89,3000-01-01,2010-06-30,2008-01-01,30,2008-09-18,2.0,24000.0


In [ ]:
inpatient[inpatient.DESYNPUF_ID == '00000B48BCF4AD29']

In [ ]:
train_dates[clean_df.DESYNPUF_ID.unique()][0]

In [ ]:
def calc_ip_amt(end_train_dt, patient_id):
    #end_train_dt = dateutil.parser.parse(str(end_train_dt)).date()
    #pd.to_datetime(x, format='%Y%m%d')
    id_mask = (inpatient.DESYNPUF_ID == patient_id)
    return inpatient[inpatient.CLM_THRU_DT<end_train_dt][id_mask].CLM_PMT_AMT.sum()

In [ ]:
clean_df.head(5)

In [ ]:
#Create variable indicating number of days from previous hospitalization
# last_hosp_date = working_df.DESYNPUF_ID.apply(lambda x: max(working_df.CLM_ADMSN_DT[working_df.DESYNPUF_ID==x]))

In [ ]:
#final = clean_df.drop_duplicates('DESYNPUF_ID')
#final.DESYNPUF_ID.duplicated().unique()

In [ ]:
final.head(5)

In [ ]:
#Write pre-selected data into a csv for cleaning and feature selection
final.to_csv('preprocessed_data.csv')